###  AIML Online Capstone Project - AUTOMATIC TICKET ASSIGNMENT (Group 4)

#### Data Pre-Processing
* This section of the Code deals with the Data Pre-Processing

In [1]:
# Import all libraries
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english')) 
from bs4 import BeautifulSoup
from string import digits
import string

In [2]:
# Define Paramaters
spl_param = 50 # Threashold to decide if the record need to be deleted
remove_email = 1 # Paramter to remove email from the text
remove_caller = 1 # parameter to remove  caller from the text
delete_null_records = 1 # paramter to delete ull records 
min_words_short = 1 # If no of words less than this delete record
min_words_total = 2 # If toatl no of words (both short and long desc) less than this delete record
copy_short_to_long = 1 # copy short desc to long if long description is empty

In [3]:
# Define Functions..
def remove_html(text):
  soup = BeautifulSoup(text,'lxml')
  html_free = soup.get_text()
  return html_free

def remove_punctuation(text):
  no_punct = "".join([c for c in text if c not in string.punctuation])
  return no_punct

def remove_digits(list1): 
    remove_digits = str.maketrans('', '', digits) 
    list1 = [i.translate(remove_digits) for i in list1] 
    return list1

def femail(desc):
    if(len(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", desc))) > 0:
      remail = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", desc)[0]
      return remail
    
def remove_space(desc):
    desc = remove_spl(desc)
    if remove_email == 1 :
        fltr_txt = ' '.join([item for item in desc.split() if '@' not in item])
    else: 
        fltr_txt = ' '.join([item for item in desc.split()])
    fltr_txt = re.sub(r"received from:","",fltr_txt)
    alpha = ""
    for char in fltr_txt:
        if char.isspace():
          alpha+= " "
        if char.isalnum():
          alpha+= char.lower()
    return alpha, len(alpha.split())

def remove_space_all(desc):
    fltr_txt = ''.join([item for item in desc.split()])
    return(fltr_txt)
    

def remove_spl(desc):
    return(desc.encode('ascii','ignore').decode())

def trimming(desc):
    desc = " ".join([item for item in desc.split() if item not in stop_words1])
    return(desc)
    
def trimmed_words_combine(desc1,desc2):
    desc = desc1
    if desc1 == desc2:
        desc = desc1
    else :
        desc = desc1+" "+desc2
    return desc

def mail_type(fmail):
    omail=fmail    
    if fmail.find("gmail") >= 0:
        omail = "gmail"
    elif fmail.find("monitoring_tool@") >=0:
        omail = "monitoring_tool"
    elif fmail.find("company") >=0:
        omail = "company"
    else :
        omail = "others"
    return(omail)

def cfind(word,string):
    flag = 0
    for i in word:
        if string.find(i) >= 0:  flag = 1
    return(flag)   

def spl_char_removed(original_txt,final_txt):
    a = (len(original_txt) - len(final_txt))*100/len(original_txt)
    return(a)

def del_null(df):
    if delete_null_records == 1 :
        print("\nNull Records: \n",df.isnull().sum())
        df_n = df.dropna()
        print("\nno of Null records Deleted :",len(df)-len(df_n))
        return(df_n)
    else : return(df)

In [4]:
# Please note that this Dataset has 4 more extra columns to identify the language and transilate to English
# Tried this transilation in Python using googletrans API but there is limitation on the numbers of calls, hence 
# googl sheets was used 
df = pd.read_excel("input_data_translated.xlsx", encoding="utf-8",dtype="str")

In [6]:
a1 = df.groupby("Assignment group")["Assignment group"].count().sort_values(ascending = False)
thr75 = 75
thr85 = 85
thr95 = 95
thr55 = 56

new_group_75 = np.round(np.cumsum(a1[np.cumsum(a1*100/8500)<=thr75]*100/8500),2).index
new_group_85 = np.round(np.cumsum(a1[np.cumsum(a1*100/8500)<=thr85]*100/8500),2).index
new_group_95 = np.round(np.cumsum(a1[np.cumsum(a1*100/8500)<=thr95]*100/8500),2).index
new_group_55 = np.round(np.cumsum(a1[np.cumsum(a1*100/8500)<=thr55]*100/8500),2).index


In [7]:
#Process the text ...

print("DataFrame Before processing:\n",df.count())
print("Shape : ",df.shape)

# Remove the caller names form the description (based on parameter). This can be done by adding the users list to the stop words
users = list(df["Caller"])
users = np.hstack(np.unique(users))
if remove_caller == 1 :
    stop_words1 = list(stop_words) + list(users)
else :
    stop_words1 = list(stop_words)

# Delete null records based on paramter
df = del_null(df)

#process text to crete trimmed words...
df["trimmed_words_short"] = remove_digits(df["short_desc_en"].apply(lambda x : remove_html(remove_punctuation(trimming(remove_space(x)[0])))))
df["trimmed_words_long"] = remove_digits(df["description_en"].apply(lambda x : remove_html(remove_punctuation(trimming(remove_space(x)[0])))))
df["trimmed_words_short"] = df["trimmed_words_short"].apply(lambda x : remove_space(x)[0])
df["trimmed_words_long"] = df["trimmed_words_long"].apply(lambda x : remove_space(x)[0])

#copy short description to long if long description is null .. based on paramter
if copy_short_to_long == 1:
    df["trimmed_words_long"] = df.apply(lambda x : x["trimmed_words_short"] if len(x["trimmed_words_long"])==0 else x["trimmed_words_long"], axis = 1)

#add new group columns to the dataframe
df["new_group_75"] =  df["Assignment group"].apply(lambda x : x if x in new_group_75 else "GRP_9999")
df["new_group_85"] =  df["Assignment group"].apply(lambda x : x if x in new_group_85 else "GRP_9999")
df["new_group_95"] =  df["Assignment group"].apply(lambda x : x if x in new_group_95 else "GRP_9999")
df["new_group_55"] =  df["Assignment group"].apply(lambda x : x if x in new_group_55 else "GRP_9999")


#mark records for delete based on parameters.. 
df["mark_for_delete"] = df.apply(lambda x : 1 if len(x["trimmed_words_short"]) < min_words_short else 
                      1 if len(x["trimmed_words_short"])+len(x["trimmed_words_long"])< min_words_total else
                      1 if spl_char_removed(x["Short description"],remove_spl(x["Short description"]))>spl_param else 0, axis = 1)

print("Data Frame After processing:\n\n",df.count())
print("Shape : ",df.shape)
print("\nno of records marked for delete : ",len(df[df["mark_for_delete"]==1]))

#Removing all the records marked for delete before further processing.....
df = df[df["mark_for_delete"]==0]
print("\nFinal Shape : ",df.shape)

df["Caller"] = df["Caller"].apply(lambda x : remove_space_all(x))

# print records having zero words
print("trimmed_words_short having 0 words :",df[df["trimmed_words_short"].apply(lambda x : True if len(x)==0 else False)]["trimmed_words_short"].count())
print("trimmed_words_long having 0 words :",df[df["trimmed_words_long"].apply(lambda x : True if len(x)==0 else False)]["trimmed_words_long"].count())


DataFrame Before processing:
 Short description    8492
Description          8499
short_desc_lang      8492
short_desc_en        8492
description_lang     8497
description_en       8497
Caller               8500
Assignment group     8500
dtype: int64
Shape :  (8500, 8)

Null Records: 
 Short description    8
Description          1
short_desc_lang      8
short_desc_en        8
description_lang     3
description_en       3
Caller               0
Assignment group     0
dtype: int64

no of Null records Deleted : 11
Data Frame After processing:

 Short description      8489
Description            8489
short_desc_lang        8489
short_desc_en          8489
description_lang       8489
description_en         8489
Caller                 8489
Assignment group       8489
trimmed_words_short    8489
trimmed_words_long     8489
new_group_75           8489
new_group_85           8489
new_group_95           8489
new_group_55           8489
mark_for_delete        8489
dtype: int64
Shape :  (8489, 15)

In [8]:
df_new = df[["trimmed_words_short","trimmed_words_long","Caller","Assignment group","new_group_75","new_group_85","new_group_95","new_group_55"]]
df_new.to_excel("processed_trimmed_data.xlsx")